In [1]:
import numpy as np
import pandas as pd

import anndata
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns
from adata_utilis import average_fraction
from interaction_functions import interaction_analysis_same

if __name__ == '__main__':
    
    print('Load interaction')
    fn_int = '/home/yike/phd/dengue/data/interaction_unpacked_mouse.tsv'
    interactions = pd.read_csv(fn_int, sep='\t')[['gene_name_a', 'gene_name_b']]
    ga, gb = interactions['gene_name_a'], interactions['gene_name_b']
    
    if True:
        print('Load high-quality cells only')
        fn_h5ad = '/home/yike/phd/dengue/data/mergedata_20200930_high_quality.h5ad'
        adata = anndata.read_h5ad(fn_h5ad)
        adata.obs['dataset'] = adata.obs['platform'].replace({
            '10X': 'child',
            'plate': 'adult'   
        })
        adata.obs['sick'] = adata.obs['Condition'].replace({
            'S_dengue': 'sick',
            'dengue': 'sick',
            'DWS': 'sick'
        })
        sc.pp.normalize_total(adata, target_sum=1e6) #normalize data to CPM (counts per million)
        
    print('Restrict to interaction genes')
    genes = np.unique(interactions)
    adatag = adata[:, genes]
    
    res = average_fraction(adatag, ['dataset', 'sick', 'cell_type'])

Load interaction
Load high-quality cells only
Restrict to interaction genes
Load interaction


/usr/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [11]:
# only healthy, interactions both expressed in adult and child
only_healthy = interaction_analysis_same('child', 'adult', 'Healthy', 'Healthy', 'B_cells', res)
cell_types = ['B_cells','NK_cells' , 'T_cells', 'Monocytes', 'Plasmablasts', 'pDCs', 'cDCs']
for cell_type in cell_types:
    only_healthy[cell_type].to_excel('/home/yike/phd/dengue/data/excels/interaction_analysis/' + cell_type + '_only_healthy.xls')

In [12]:
# only sick, interactions both expressed in adult and child
only_sick = interaction_analysis_same('child', 'adult', 'sick', 'sick', 'B_cells', res)
cell_types = ['B_cells','NK_cells' , 'T_cells', 'Monocytes', 'Plasmablasts', 'pDCs', 'cDCs']
for cell_type in cell_types:
    only_sick[cell_type].to_excel('/home/yike/phd/dengue/data/excels/interaction_analysis/' + cell_type + '_only_sick.xls')